# Advance SQL

## Table Relationships & Keys

### 🔸 What is a Primary Key?
- Uniquely identifies each row.
- Cannot be NULL.
- Ensures each row is independently addressable.

In [ ]:
CREATE TABLE customer (
  customer_id SERIAL PRIMARY KEY,
  name TEXT NOT NULL
);

### 🔸 What is a Foreign Key?
- A column that references another table’s primary key.
- Ensures relational integrity between tables.

In [ ]:
CREATE TABLE rental (
  rental_id SERIAL PRIMARY KEY,
  customer_id INT REFERENCES customer(customer_id)
);

### 🔸 Types of Relationships
| Type             | Example                                | Use Case                              |
|------------------|----------------------------------------|----------------------------------------|
| One-to-One       | user → profile                         | Each user has one profile              |
| One-to-Many      | customer → rental                      | A customer can have many rentals       |
| Many-to-Many     | film ↔ actor (via film_actor)          | A film has many actors, actors in many films |

In [ ]:
CREATE TABLE film_actor (
  film_id INT REFERENCES film(film_id),
  actor_id INT REFERENCES actor(actor_id),
  PRIMARY KEY (film_id, actor_id)
);

## Joining Tables

### 🔸 Sample Tables

#### Table A: `customers`

In [ ]:
CREATE TABLE customers (
  id INT,
  name TEXT
);

INSERT INTO customers (id, name) VALUES
(1, 'Alice'),
(2, 'Bob'),
(3, 'Charlie');

#### Table B: `orders`

In [ ]:
CREATE TABLE orders (
  id INT,
  customer_id INT,
  product TEXT
);

INSERT INTO orders (id, customer_id, product) VALUES
(101, 1, 'Book'),
(102, 2, 'Pen'),
(103, 4, 'Notebook');

### 🔸 INNER JOIN
**Only matching rows in both tables (customer exists and made order)**

![Inner Join Diagram](images/inner.png)

In [ ]:
SELECT c.id, c.name, o.product
FROM customers AS c
INNER JOIN orders o ON c.id = o.customer_id;

### 🔸 LEFT JOIN
**All customers, with orders if they exist**

![Left Join Diagram](images/left.png)

In [ ]:
SELECT c.id, c.name, o.product
FROM customers c
LEFT JOIN orders o ON c.id = o.customer_id;

### 🔸 RIGHT JOIN
**All orders, with customers if they exist**

![Right Join Diagram](images/right.png)

In [ ]:
SELECT c.id, c.name, o.product
FROM customers c
RIGHT JOIN orders o ON c.id = o.customer_id;

### 🔸 FULL OUTER JOIN
**All customers and all orders, match when possible**

![Full Outer Join Diagram](images/full.png)

In [ ]:
SELECT c.id, c.name, o.product
FROM customers c
FULL OUTER JOIN orders o ON c.id = o.customer_id;

### 🔸 CROSS JOIN
**All possible combinations of customers and orders**

![Cross Join Diagram](images/cross.png)

In [ ]:
SELECT c.name, o.product
FROM customers c
CROSS JOIN orders o;

## UNION and UNION ALL

### 🔸 What is UNION?
Combines results of two SELECT queries and removes duplicates.

In [ ]:
SELECT first_name FROM customer
UNION
SELECT name FROM staff;

### 🔸 What is UNION ALL?
Combines results and **includes duplicates**.

In [ ]:
SELECT first_name FROM customer
UNION ALL
SELECT name FROM staff;

### 📝 Rules for UNION and UNION ALL:
- Number of columns must match
- Data types should be compatible
- `UNION` removes duplicates, `UNION ALL` is faster and keeps all rows

## INSERT, UPDATE, DELETE

### 🔸 INSERT

In [ ]:
INSERT INTO customer (first_name, last_name) VALUES ('Ali', 'Rahimi');

### 🔸 Multi-row INSERT

In [ ]:
INSERT INTO staff (first_name, last_name)
VALUES ('Sara', 'Miri'), ('Mehdi', 'Shahi');

### 🔸 UPDATE

In [ ]:
UPDATE customer SET email = 'x@x.com' WHERE customer_id = 1;

### 🔸 DELETE

In [ ]:
DELETE FROM customer WHERE customer_id = 10;

## Table Management

### 🔸 PostgreSQL Data Types

| Type          | Description                                  | Example                         |
|---------------|----------------------------------------------|----------------------------------|
| `SMALLINT`    | Small integers (-32k to 32k)                 | `age SMALLINT`                  |
| `INTEGER`     | Standard integers                            | `count INTEGER`                 |
| `SERIAL`      | Auto-incrementing integer                    | `id SERIAL PRIMARY KEY`         |
| `VARCHAR(n)`  | Variable-length string with limit            | `email VARCHAR(100)`            |
| `CHAR(n)`     | Fixed-length string                          | `country_code CHAR(2)`          |
| `TEXT`        | Unlimited-length string                      | `description TEXT`              |
| `BOOLEAN`     | TRUE, FALSE, NULL                            | `is_active BOOLEAN`             |
| `NUMERIC(p,s)`| Exact decimal (e.g. money)                   | `price NUMERIC(10, 2)`          |
| `DATE`        | Calendar date                                | `birth_date DATE`               |
| `TIMESTAMP`   | Date and time                                | `created_at TIMESTAMP`          |
| `UUID`        | Unique identifier                            | `token UUID`                    |
| `JSONB`       | Binary JSON storage                          | `settings JSONB`                |
| `ARRAY`       | PostgreSQL arrays                            | `tags TEXT[]`                   |

In [ ]:
CREATE TYPE status AS ENUM ('active', 'inactive', 'banned');

### 🔸 Creating Tables with Constraints

In [ ]:
CREATE TABLE employee (
  id SERIAL PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  email TEXT UNIQUE,
  department_id INT REFERENCES department(id)
);

### 🔸 NULL vs NOT NULL
- `NOT NULL`: must be filled (e.g., critical fields like `email`).
- `NULL`: optional (e.g., `comment`).

### 🔸 ALTER TABLE

In [ ]:
ALTER TABLE employee ADD COLUMN salary NUMERIC;

In [ ]:
ALTER TABLE employee ALTER COLUMN salary TYPE INTEGER;

In [ ]:
ALTER TABLE employee DROP COLUMN salary;

### 🔸 RENAME TABLE / COLUMN

In [ ]:
ALTER TABLE employee RENAME TO staff;

In [ ]:
ALTER TABLE staff RENAME COLUMN name TO full_name;

## Views and Virtual Tables

**Why use views?**
- 🧩 Abstract complexity
- 🔐 Enforce access control
- 🔄 Reusability for reporting or repeated queries

In [ ]:
CREATE VIEW active_customers AS
SELECT customer_id, first_name, last_name
FROM customer
WHERE active = TRUE;

In [ ]:
SELECT * FROM active_customers;

In [ ]:
DROP VIEW active_customers;

## Logical Delete (Soft Delete)
Instead of physically removing data, mark it as deleted using a flag column like `is_deleted`.

In [ ]:
ALTER TABLE customer ADD COLUMN is_deleted BOOLEAN DEFAULT FALSE;

In [ ]:
UPDATE customer SET is_deleted = TRUE WHERE customer_id = 12;

## Self-Relations: Hierarchies and Recursion
A self-relation is when a row relates to another row in the **same table**, like employees and their managers.

In [ ]:
CREATE TABLE employee (
  id SERIAL PRIMARY KEY,
  name TEXT,
  manager_id INT REFERENCES employee(id)
);

### 🔸 Recursive Query (WITH RECURSIVE)
Query a chain of management hierarchy using Common Table Expressions.

In [ ]:
WITH RECURSIVE management_chain AS (
  SELECT id, name, manager_id FROM employee WHERE manager_id IS NULL
  UNION ALL
  SELECT e.id, e.name, e.manager_id
  FROM employee e
  JOIN management_chain mc ON e.manager_id = mc.id
)
SELECT * FROM management_chain;

## Indexes

### 🔸 Why Use Indexes?
- Speed up search & joins
- Reduce full table scans

In [ ]:
CREATE INDEX idx_customer_email ON customer(email);

In [ ]:
EXPLAIN ANALYZE SELECT * FROM customer WHERE email = 'x@x.com';

## Normalization

### 🔸 Step-by-step Normalization Example
**Unnormalized Table:**

In [ ]:
CREATE TABLE orders (
  order_id INT,
  customer_name TEXT,
  items TEXT -- 'pen, paper, notebook'
);

**1NF:** Remove repeated items

In [ ]:
CREATE TABLE order_item (
  order_id INT,
  item TEXT
);

**2NF:** Remove partial dependencies

In [ ]:
CREATE TABLE customer (
  id SERIAL PRIMARY KEY,
  name TEXT
);
ALTER TABLE orders ADD COLUMN customer_id INT REFERENCES customer(id);

**3NF:** Remove transitive dependencies

In [ ]:
CREATE TABLE item (
  id SERIAL PRIMARY KEY,
  name TEXT,
  price NUMERIC
);
CREATE TABLE order_item (
  order_id INT,
  item_id INT REFERENCES item(id)
);

## Python psycopg2 and ORMs

In [ ]:
import psycopg2
conn = psycopg2.connect(
  dbname='dvdrental', user='postgres', password='yourpass', host='localhost')
cur = conn.cursor()
cur.execute('SELECT * FROM customer LIMIT 5;')
for row in cur.fetchall():
    print(row)
cur.close()
conn.close()